In [1]:
import rosbag2_py
from rclpy.serialization import serialize_message
import std_msgs.msg as std_msgs
import numpy as np
from builtin_interfaces.msg import Time
import ros2_numpy as rnpy
from geometry_msgs.msg import TransformStamped
from pyquaternion import Quaternion

In [2]:
from waymo_open_dataset import v2
import matplotlib.pyplot as plt
import tensorflow as tf
import dask.dataframe as dd

import warnings
# Disable annoying warnings from PyArrow using under the hood.
warnings.simplefilter(action='ignore', category=FutureWarning)

2023-03-29 17:26:08.618763: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
# Path to the directory with all components
dataset_dir = '/home/myron/data/Waymo/data'
context = '10504764403039842352_460_000_480_000'

lidar_df = dd.read_parquet(f'{dataset_dir}/lidar/testing_lidar_{context}.parquet')
lidar_calib_df = dd.read_parquet(f'{dataset_dir}/lidar_calibration/testing_lidar_calibration_{context}.parquet')
# lidar_pose_df = dd.read_parquet(f'{dataset_dir}/lidar_pose/testing_lidar_pose_{context}.parquet')
vehicle_pose_df = dd.read_parquet(f'{dataset_dir}/vehicle_pose/testing_vehicle_pose_10504764403039842352_460_000_480_000.parquet')

In [4]:
lidar_lcalib_df = v2.merge(lidar_df, lidar_calib_df, right_group=True)
# lidar_lcalib_lpose_df = v2.merge(lidar_pose_df, lidar_lcalib_df, right_group=True)
lidar_lcalib_vpose_df = v2.merge(vehicle_pose_df, lidar_lcalib_df, right_group=False)

In [5]:
lidar_lcalib_vpose_df.head()

,key.segment_context_name,key.frame_timestamp_micros,[VehiclePoseComponent].world_from_vehicle.transform,key.laser_name,[LiDARComponent].range_image_return1.values,[LiDARComponent].range_image_return1.shape,[LiDARComponent].range_image_return2.values,[LiDARComponent].range_image_return2.shape,[LiDARCalibrationComponent].extrinsic.transform,[LiDARCalibrationComponent].beam_inclination.min,[LiDARCalibrationComponent].beam_inclination.max,[LiDARCalibrationComponent].beam_inclination.values
0,10504764403039842352_460_000_480_000,1558060923374527,"[0.015530851147822279, 0.9993379682253831, -0....",1,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[64, 2650, 4]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[64, 2650, 4]","[-0.8531144853352591, -0.5217225826529568, 0.0...",-0.315786,0.039599,"[-0.3105449854411111, -0.30006279157785287, -0..."
1,10504764403039842352_460_000_480_000,1558060923374527,"[0.015530851147822279, 0.9993379682253831, -0....",2,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[200, 600, 4]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[200, 600, 4]","[0.9997788317633413, -0.020411114732822817, 0....",-1.570796,0.523599,None
2,10504764403039842352_460_000_480_000,1558060923374527,"[0.015530851147822279, 0.9993379682253831, -0....",3,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[200, 600, 4]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[200, 600, 4]","[0.005220212948586411, -0.9999596896584837, 0....",-1.570796,0.523599,None
3,10504764403039842352_460_000_480_000,1558060923374527,"[0.015530851147822279, 0.9993379682253831, -0....",4,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[200, 600, 4]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[200, 600, 4]","[0.012464598372826827, 0.9999033927187172, -0....",-1.570796,0.523599,None
4,10504764403039842352_460_000_480_000,1558060923374527,"[0.015530851147822279, 0.9993379682253831, -0....",5,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[200, 600, 4]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[200, 600, 4]","[-0.9996589391821362, -0.012444893206838104, -...",-1.570796,0.523599,None


In [6]:
class RosbagWriter:
    def __init__(self, topic_msg_type_map: dict, output_bag_path: str, serialization='cdr') -> None:
        self.writer = rosbag2_py.SequentialWriter()
        self.open_bag_for_writing(output_bag_path, serialization)
        for topic, msg_type in topic_msg_type_map.items():
            self.writer.create_topic(rosbag2_py._storage.TopicMetadata(
                name=topic,
                type=msg_type,
                serialization_format=serialization))
    
    def open_bag_for_writing(self, path, serialization_format='cdr'):
        storage_options = rosbag2_py.StorageOptions(uri=path, storage_id='sqlite3')

        converter_options = rosbag2_py.ConverterOptions(
            input_serialization_format=serialization_format,
            output_serialization_format=serialization_format)

        self.writer.open(storage_options, converter_options)
    
    def write(self, topic: str, ros2_msg: any, timestamp_ns: int):
        self.writer.write(topic, serialize_message(ros2_msg), timestamp_ns)

In [7]:
bag_out_path = f'output/waymo_{context}'
rb_writer = RosbagWriter({'/points': 'sensor_msgs/msg/PointCloud2',
                          '/base_link': 'geometry_msgs/msg/TransformStamped',
                          '/lidar': 'geometry_msgs/msg/TransformStamped'}, bag_out_path)

[INFO] [1680107172.251148730] [rosbag2_storage]: Opened database 'output/waymo_10504764403039842352_460_000_480_000/waymo_10504764403039842352_460_000_480_000_0.db3' for READ_WRITE.


In [8]:
def time_msg_from_timestamp_us(timestamp_us: int):
    return Time(sec=int(timestamp_us/1e6), nanosec=int((timestamp_us%1e6)*1e3))

In [9]:
def lidar_range_img_to_pcd_xyzi_msg(range_img, lidar_calib, lidar_frame: str, time_msg: Time):
    pc = v2.convert_range_image_to_point_cloud(range_img, lidar_calib, keep_polar_features=True)
    pc = np.rec.fromarrays(tf.transpose(tf.gather(pc, [3,4,5,1], axis=1)),
                           dtype={'names': ('x', 'y', 'z', 'i'),
                                  'formats': (np.float32, np.float32, np.float32, np.float32)})
    return rnpy.array_to_pointcloud2(pc, time_msg, lidar_frame)

In [10]:
def make_transform_msg(tf_matrix, from_frame_id: str, to_frame_id: str, time_msg: Time):
  t = TransformStamped()

  t.header.stamp = time_msg
  t.header.frame_id = from_frame_id
  t.child_frame_id = to_frame_id
  t.transform.translation.x = float(tf_matrix[0, 3])
  t.transform.translation.y = float(tf_matrix[1, 3])
  t.transform.translation.z = float(tf_matrix[2, 3])
  quat = Quaternion(matrix=tf_matrix)
  t.transform.rotation.x = float(quat.x)
  t.transform.rotation.y = float(quat.y)
  t.transform.rotation.z = float(quat.z)
  t.transform.rotation.w = float(quat.w)
  return t

In [11]:
for i, (_, r) in enumerate(lidar_lcalib_vpose_df.iterrows()):
  # Create component dataclasses for the raw data
  # print(r)
  lidar = v2.LiDARComponent.from_dict(r)
  laser_name = int(lidar.key.laser_name)
  

  if(laser_name == 1):
    timestamp_us = lidar.key.frame_timestamp_micros
    time_msg = time_msg_from_timestamp_us(timestamp_us)
    
    lidar_calib = v2.LiDARCalibrationComponent.from_dict(r)
    vehicle_pose_df = v2.VehiclePoseComponent.from_dict(r)
    vehicle_pose = np.array(vehicle_pose_df.world_from_vehicle.transform).reshape((4,4))
    lidar_pose = np.array(lidar_calib.extrinsic.transform).reshape((4,4))
    
    pcd_msg = lidar_range_img_to_pcd_xyzi_msg(lidar.range_image_return1, lidar_calib, '/lidar', time_msg)
    lidar_tf = make_transform_msg(lidar_pose, '/base_link', '/lidar', time_msg)
    vehicle_tf = make_transform_msg(vehicle_pose, '/map', '/base_link', time_msg)
    time_ns = int(timestamp_us*1e3)
    rb_writer.write('/points', pcd_msg, time_ns)
    rb_writer.write('/lidar', lidar_tf, time_ns)
    rb_writer.write('/base_link', vehicle_tf, time_ns)

2023-03-29 17:26:14.497287: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
del rb_writer